In [1]:
import requests
import pandas as pd
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
api_key = os.getenv('FOOTBALL_DATA_KEY')
base_url = 'https://api.football-data.org/v4'
headers = {'X-Auth-Token': api_key}

In [3]:
def get_premier_league_matches(season: str) -> dict:
    url = f'{base_url}/competitions/PL/matches?season={season}&status=FINISHED'
    response = requests.get(url, headers=headers)
    data = response.json()
    return data['matches']

In [4]:
def calculate_home_points(row):
    if row['home_score'] > row['away_score']:
        return 3
    elif row['home_score'] == row['away_score']:
        return 1
    else:
        return 0


def calculate_away_points(row):
    if row['home_score'] < row['away_score']:
        return 3
    elif row['home_score'] == row['away_score']:
        return 1
    else:
        return 0


result_mapping = {'HOME_TEAM': 0, 'DRAW': 1, 'AWAY_TEAM': 2}

# 1 Season

In [5]:
matches = get_premier_league_matches(season='2023')
match_data = []
for match in matches:
    match_data.append({
        'date': match['utcDate'],
        'home_team': match['homeTeam']['shortName'],
        'away_team': match['awayTeam']['shortName'],
        'home_score': match['score']['fullTime']['home'],
        'away_score': match['score']['fullTime']['away'],
        'result': match['score']['winner'],
        'matchday': match['matchday']
    })

df = pd.DataFrame(match_data)
df = df.sort_values(by='date', ascending=True)
df['date'] = pd.to_datetime(df['date'])

In [6]:
df

,date,home_team,away_team,home_score,away_score,result,matchday
0,2023-08-11 19:00:00+00:00,Burnley,Man City,0,3,AWAY_TEAM,1
1,2023-08-12 12:00:00+00:00,Arsenal,Nottingham,2,1,HOME_TEAM,1
2,2023-08-12 14:00:00+00:00,Bournemouth,West Ham,1,1,DRAW,1
3,2023-08-12 14:00:00+00:00,Brighton Hove,Luton Town,4,1,HOME_TEAM,1
4,2023-08-12 14:00:00+00:00,Everton,Fulham,0,1,AWAY_TEAM,1
...,...,...,...,...,...,...,...
372,2024-05-19 15:00:00+00:00,Brighton Hove,Man United,0,2,AWAY_TEAM,38
371,2024-05-19 15:00:00+00:00,Brentford,Newcastle,2,4,AWAY_TEAM,38
370,2024-05-19 15:00:00+00:00,Arsenal,Everton,2,1,HOME_TEAM,38
373,2024-05-19 15:00:00+00:00,Burnley,Nottingham,1,2,AWAY_TEAM,38


In [7]:
df.head(2)

,date,home_team,away_team,home_score,away_score,result,matchday
0,2023-08-11 19:00:00+00:00,Burnley,Man City,0,3,AWAY_TEAM,1
1,2023-08-12 12:00:00+00:00,Arsenal,Nottingham,2,1,HOME_TEAM,1


In [8]:
df['home_points'] = df.apply(calculate_home_points, axis=1)
df['away_points'] = df.apply(calculate_away_points, axis=1)
df['result'] = df['result'].map(result_mapping)

In [9]:
df.head(2)

,date,home_team,away_team,home_score,away_score,result,matchday,home_points,away_points
0,2023-08-11 19:00:00+00:00,Burnley,Man City,0,3,2,1,0,3
1,2023-08-12 12:00:00+00:00,Arsenal,Nottingham,2,1,0,1,3,0


In [10]:
# Calculate rolling window stats
home_df = df[['date', 'home_team', 'home_score', 'away_score', 'home_points']].copy()
home_df.columns = ['date', 'team', 'goals_for', 'goals_against', 'points']

away_df = df[['date', 'away_team', 'away_score', 'home_score', 'away_points']].copy()
away_df.columns = ['date', 'team', 'goals_for', 'goals_against', 'points']

combined_df = pd.concat([home_df, away_df], ignore_index=True)

combined_df = combined_df.sort_values(by=['team', 'date']).reset_index(drop=True)

combined_df['form_last_5'] = combined_df.groupby('team')['points'].shift(
    1).rolling(window=5).sum().reset_index(0, drop=True)

combined_df['avg_goals_for_last_5'] = combined_df.groupby(
    'team')['goals_for'].shift(1).rolling(window=5).mean().reset_index(0, drop=True)

combined_df['avg_goals_against_last_5'] = combined_df.groupby(
    'team')['goals_against'].shift(1).rolling(window=5).mean().reset_index(0, drop=True)

combined_df

,date,team,goals_for,goals_against,points,form_last_5,avg_goals_for_last_5,avg_goals_against_last_5
0,2023-08-12 12:00:00+00:00,Arsenal,2,1,3,NaN,NaN,NaN
1,2023-08-21 19:00:00+00:00,Arsenal,1,0,3,NaN,NaN,NaN
2,2023-08-26 14:00:00+00:00,Arsenal,2,2,1,NaN,NaN,NaN
3,2023-09-03 15:30:00+00:00,Arsenal,3,1,3,NaN,NaN,NaN
4,2023-09-17 15:30:00+00:00,Arsenal,1,0,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
755,2024-04-24 18:45:00+00:00,Wolverhampton,0,1,0,2.0,0.8,1.8
756,2024-04-27 14:00:00+00:00,Wolverhampton,2,1,3,2.0,0.8,1.6
757,2024-05-04 16:30:00+00:00,Wolverhampton,1,5,0,4.0,1.0,1.6
758,2024-05-11 14:00:00+00:00,Wolverhampton,1,3,0,4.0,1.0,2.2


In [11]:
home_stats = combined_df[['date', 'team', 'form_last_5', 'avg_goals_for_last_5', 'avg_goals_against_last_5']].copy()
away_stats = home_stats.copy()

In [12]:
# Merge home team stats back to the original DataFrame

df = pd.merge(df, home_stats, how='left', left_on=['home_team', 'date'], right_on=['team', 'date'])
df = df.rename(columns={
    'form_last_5': 'home_team_form_last_5',
    'avg_goals_for_last_5': 'home_team_avg_goals_for_last_5',
    'avg_goals_against_last_5': 'home_team_avg_goals_against_last_5'
})

# Merge away team stats back to the original DataFrame
df = pd.merge(df, away_stats, how='left', left_on=['away_team', 'date'], right_on=['team', 'date'])
df = df.rename(columns={
    'form_last_5': 'away_team_form_last_5',
    'avg_goals_for_last_5': 'away_team_avg_goals_for_last_5',
    'avg_goals_against_last_5': 'away_team_avg_goals_against_last_5'
})

In [13]:
df

,date,home_team,away_team,home_score,away_score,result,matchday,home_points,away_points,team_x,home_team_form_last_5,home_team_avg_goals_for_last_5,home_team_avg_goals_against_last_5,team_y,away_team_form_last_5,away_team_avg_goals_for_last_5,away_team_avg_goals_against_last_5
0,2023-08-11 19:00:00+00:00,Burnley,Man City,0,3,2,1,0,3,Burnley,NaN,NaN,NaN,Man City,NaN,NaN,NaN
1,2023-08-12 12:00:00+00:00,Arsenal,Nottingham,2,1,0,1,3,0,Arsenal,NaN,NaN,NaN,Nottingham,NaN,NaN,NaN
2,2023-08-12 14:00:00+00:00,Bournemouth,West Ham,1,1,1,1,1,1,Bournemouth,NaN,NaN,NaN,West Ham,NaN,NaN,NaN
3,2023-08-12 14:00:00+00:00,Brighton Hove,Luton Town,4,1,0,1,3,0,Brighton Hove,NaN,NaN,NaN,Luton Town,NaN,NaN,NaN
4,2023-08-12 14:00:00+00:00,Everton,Fulham,0,1,2,1,0,3,Everton,NaN,NaN,NaN,Fulham,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2024-05-19 15:00:00+00:00,Brighton Hove,Man United,0,2,2,38,0,3,Brighton Hove,4.0,0.6,2.0,Man United,7.0,1.6,2.0
376,2024-05-19 15:00:00+00:00,Brentford,Newcastle,2,4,2,38,0,3,Brentford,10.0,1.8,0.6,Newcastle,7.0,2.4,1.6
377,2024-05-19 15:00:00+00:00,Arsenal,Everton,2,1,0,38,3,0,Arsenal,15.0,2.8,0.4,Everton,13.0,1.4,0.2
378,2024-05-19 15:00:00+00:00,Burnley,Nottingham,1,2,2,38,0,3,Burnley,5.0,1.6,1.8,Nottingham,4.0,1.4,2.0


In [14]:
# Check stats are good for Liverpool
df.loc[(df['home_team'] == 'Liverpool') | (df['away_team'] == 'Liverpool')]

,date,home_team,away_team,home_score,away_score,result,matchday,home_points,away_points,team_x,home_team_form_last_5,home_team_avg_goals_for_last_5,home_team_avg_goals_against_last_5,team_y,away_team_form_last_5,away_team_avg_goals_for_last_5,away_team_avg_goals_against_last_5
8,2023-08-13 15:30:00+00:00,Chelsea,Liverpool,1,1,1,1,1,1,Chelsea,NaN,NaN,NaN,Liverpool,NaN,NaN,NaN
12,2023-08-19 14:00:00+00:00,Liverpool,Bournemouth,3,1,0,2,3,0,Liverpool,NaN,NaN,NaN,Bournemouth,NaN,NaN,NaN
28,2023-08-27 15:30:00+00:00,Newcastle,Liverpool,1,2,2,3,0,3,Newcastle,NaN,NaN,NaN,Liverpool,NaN,NaN,NaN
37,2023-09-03 13:00:00+00:00,Liverpool,Aston Villa,3,0,0,4,3,0,Liverpool,NaN,NaN,NaN,Aston Villa,NaN,NaN,NaN
39,2023-09-16 11:30:00+00:00,Wolverhampton,Liverpool,1,3,2,5,0,3,Wolverhampton,NaN,NaN,NaN,Liverpool,NaN,NaN,NaN
57,2023-09-24 13:00:00+00:00,Liverpool,West Ham,3,1,0,6,3,0,Liverpool,13.0,2.4,0.8,West Ham,10.0,2.0,1.4
66,2023-09-30 16:30:00+00:00,Tottenham,Liverpool,2,1,0,7,3,0,Tottenham,13.0,2.6,1.0,Liverpool,15.0,2.8,0.8
76,2023-10-08 13:00:00+00:00,Brighton Hove,Liverpool,2,2,1,8,1,1,Brighton Hove,9.0,2.2,2.4,Liverpool,12.0,2.4,1.0
80,2023-10-21 11:30:00+00:00,Liverpool,Everton,2,0,0,9,3,0,Liverpool,10.0,2.4,1.2,Everton,7.0,1.8,1.2
98,2023-10-29 14:00:00+00:00,Liverpool,Nottingham,3,0,0,10,3,0,Liverpool,10.0,2.2,1.2,Nottingham,4.0,0.8,1.2


In [15]:
# drop unused columns
df = df.drop(columns=['team_x', 'team_y', 'home_points', 'away_points', 'home_score', 'away_score'], axis=1)

In [16]:
df

,date,home_team,away_team,result,matchday,home_team_form_last_5,home_team_avg_goals_for_last_5,home_team_avg_goals_against_last_5,away_team_form_last_5,away_team_avg_goals_for_last_5,away_team_avg_goals_against_last_5
0,2023-08-11 19:00:00+00:00,Burnley,Man City,2,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-12 12:00:00+00:00,Arsenal,Nottingham,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-12 14:00:00+00:00,Bournemouth,West Ham,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-12 14:00:00+00:00,Brighton Hove,Luton Town,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-12 14:00:00+00:00,Everton,Fulham,2,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
375,2024-05-19 15:00:00+00:00,Brighton Hove,Man United,2,38,4.0,0.6,2.0,7.0,1.6,2.0
376,2024-05-19 15:00:00+00:00,Brentford,Newcastle,2,38,10.0,1.8,0.6,7.0,2.4,1.6
377,2024-05-19 15:00:00+00:00,Arsenal,Everton,0,38,15.0,2.8,0.4,13.0,1.4,0.2
378,2024-05-19 15:00:00+00:00,Burnley,Nottingham,2,38,5.0,1.6,1.8,4.0,1.4,2.0


In [17]:
df = pd.get_dummies(df, columns=['home_team', 'away_team'], dtype=int)

In [18]:
df = df.dropna()
df

,date,result,matchday,home_team_form_last_5,home_team_avg_goals_for_last_5,home_team_avg_goals_against_last_5,away_team_form_last_5,away_team_avg_goals_for_last_5,away_team_avg_goals_against_last_5,home_team_Arsenal,...,away_team_Liverpool,away_team_Luton Town,away_team_Man City,away_team_Man United,away_team_Newcastle,away_team_Nottingham,away_team_Sheffield Utd,away_team_Tottenham,away_team_West Ham,away_team_Wolverhampton
49,2023-09-23 14:00:00+00:00,1,6,7.0,1.2,1.4,7.0,1.0,2.0,0,...,0,0,0,0,0,0,0,0,0,0
51,2023-09-23 14:00:00+00:00,0,6,15.0,2.8,0.6,7.0,1.4,1.4,0,...,0,0,0,0,0,1,0,0,0,0
52,2023-09-23 16:30:00+00:00,2,6,6.0,1.6,1.2,1.0,0.4,1.8,0,...,0,0,0,0,0,0,0,0,0,0
54,2023-09-24 13:00:00+00:00,1,6,13.0,1.8,0.8,13.0,2.6,1.0,1,...,0,0,0,0,0,0,0,1,0,0
55,2023-09-24 13:00:00+00:00,0,6,12.0,3.0,1.4,3.0,0.8,1.6,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2024-05-19 15:00:00+00:00,2,38,4.0,0.6,2.0,7.0,1.6,2.0,0,...,0,0,0,1,0,0,0,0,0,0
376,2024-05-19 15:00:00+00:00,2,38,10.0,1.8,0.6,7.0,2.4,1.6,0,...,0,0,0,0,1,0,0,0,0,0
377,2024-05-19 15:00:00+00:00,0,38,15.0,2.8,0.4,13.0,1.4,0.2,1,...,0,0,0,0,0,0,0,0,0,0
378,2024-05-19 15:00:00+00:00,2,38,5.0,1.6,1.8,4.0,1.4,2.0,0,...,0,0,0,0,0,1,0,0,0,0


# Multiple Seasons

In [19]:
seasons=['2022', '2023']
df = pd.DataFrame()
match_data = []
for season in seasons:
    matches = get_premier_league_matches(season=season)
    for match in matches:
        match_data.append({
            'date': match['utcDate'],
            'home_team': match['homeTeam']['shortName'],
            'away_team': match['awayTeam']['shortName'],
            'home_score': match['score']['fullTime']['home'],
            'away_score': match['score']['fullTime']['away'],
            'result': match['score']['winner'],
            'matchday': match['matchday']
        })

df = pd.DataFrame(match_data)

In [20]:
df

,date,home_team,away_team,home_score,away_score,result,matchday
0,2022-08-05T19:00:00Z,Crystal Palace,Arsenal,0,2,AWAY_TEAM,1
1,2022-08-06T11:30:00Z,Fulham,Liverpool,2,2,DRAW,1
2,2022-08-06T14:00:00Z,Tottenham,Southampton,4,1,HOME_TEAM,1
3,2022-08-06T14:00:00Z,Newcastle,Nottingham,2,0,HOME_TEAM,1
4,2022-08-06T14:00:00Z,Leeds United,Wolverhampton,2,1,HOME_TEAM,1
...,...,...,...,...,...,...,...
755,2024-05-19T15:00:00Z,Crystal Palace,Aston Villa,5,0,HOME_TEAM,38
756,2024-05-19T15:00:00Z,Liverpool,Wolverhampton,2,0,HOME_TEAM,38
757,2024-05-19T15:00:00Z,Luton Town,Fulham,2,4,AWAY_TEAM,38
758,2024-05-19T15:00:00Z,Man City,West Ham,3,1,HOME_TEAM,38


In [21]:
df['home_points'] = df.apply(calculate_home_points, axis=1)
df['away_points'] = df.apply(calculate_away_points, axis=1)
df['result'] = df['result'].map(result_mapping)

home_df = df[['date', 'home_team', 'home_score', 'away_score', 'home_points']].copy()
home_df.columns = ['date', 'team', 'goals_for', 'goals_against', 'points']

away_df = df[['date', 'away_team', 'away_score', 'home_score', 'away_points']].copy()
away_df.columns = ['date', 'team', 'goals_for', 'goals_against', 'points']

combined_df = pd.concat([home_df, away_df], ignore_index=True)

combined_df = combined_df.sort_values(by=['team', 'date']).reset_index(drop=True)

combined_df['form_last_5'] = combined_df.groupby('team')['points'].shift(
    1).rolling(window=5).sum().reset_index(0, drop=True)

combined_df['avg_goals_for_last_5'] = combined_df.groupby(
    'team')['goals_for'].shift(1).rolling(window=5).mean().reset_index(0, drop=True)

combined_df['avg_goals_against_last_5'] = combined_df.groupby(
    'team')['goals_against'].shift(1).rolling(window=5).mean().reset_index(0, drop=True)

home_stats = combined_df[['date', 'team', 'form_last_5', 'avg_goals_for_last_5', 'avg_goals_against_last_5']].copy()
away_stats = home_stats.copy()

df = pd.merge(df, home_stats, how='left', left_on=['home_team', 'date'], right_on=['team', 'date'])
df = df.rename(columns={
    'form_last_5': 'home_team_form_last_5',
    'avg_goals_for_last_5': 'home_team_avg_goals_for_last_5',
    'avg_goals_against_last_5': 'home_team_avg_goals_against_last_5'
})

# Merge away team stats back to the original DataFrame
df = pd.merge(df, away_stats, how='left', left_on=['away_team', 'date'], right_on=['team', 'date'])
df = df.rename(columns={
    'form_last_5': 'away_team_form_last_5',
    'avg_goals_for_last_5': 'away_team_avg_goals_for_last_5',
    'avg_goals_against_last_5': 'away_team_avg_goals_against_last_5'
})

df = df.drop(columns=['team_x', 'team_y', 'home_points', 'away_points', 'home_score', 'away_score'], axis=1)

df = df.dropna()

# df = pd.get_dummies(df, columns=['home_team', 'away_team'], dtype=int)

In [22]:
df

,date,home_team,away_team,result,matchday,home_team_form_last_5,home_team_avg_goals_for_last_5,home_team_avg_goals_against_last_5,away_team_form_last_5,away_team_avg_goals_for_last_5,away_team_avg_goals_against_last_5
50,2022-09-03T11:30:00Z,Everton,Liverpool,1,6,3.0,0.8,1.2,8.0,3.0,1.2
51,2022-09-03T14:00:00Z,Wolverhampton,Southampton,0,6,3.0,0.4,0.8,7.0,1.4,1.8
52,2022-09-03T14:00:00Z,Tottenham,Fulham,0,6,11.0,2.0,0.8,8.0,1.6,1.4
53,2022-09-03T14:00:00Z,Nottingham,Bournemouth,2,6,4.0,0.4,2.2,4.0,0.4,3.2
54,2022-09-03T14:00:00Z,Newcastle,Crystal Palace,1,6,6.0,1.4,1.2,5.0,1.4,1.8
...,...,...,...,...,...,...,...,...,...,...,...
755,2024-05-19T15:00:00Z,Crystal Palace,Aston Villa,0,38,13.0,3.0,0.8,8.0,2.0,1.4
756,2024-05-19T15:00:00Z,Liverpool,Wolverhampton,0,38,8.0,2.4,2.0,3.0,0.8,2.4
757,2024-05-19T15:00:00Z,Luton Town,Fulham,2,38,1.0,1.0,3.2,5.0,0.8,1.6
758,2024-05-19T15:00:00Z,Man City,West Ham,0,38,15.0,3.4,0.2,4.0,1.4,3.0


In [23]:
df.columns

Index(['date', 'home_team', 'away_team', 'result', 'matchday',
       'home_team_form_last_5', 'home_team_avg_goals_for_last_5',
       'home_team_avg_goals_against_last_5', 'away_team_form_last_5',
       'away_team_avg_goals_for_last_5', 'away_team_avg_goals_against_last_5'],
      dtype='object')